In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import regex as re
import os

In [12]:
df = pd.read_excel('globalterrorismdb_0522dist.xlsx')
df.head()

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaT,58,Dominican Republic,2,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaT,130,Mexico,1,...,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaT,160,Philippines,5,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaT,78,Greece,8,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
4,197001000003,1970,1,0,NaN,0,NaT,101,Japan,4,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209706 entries, 0 to 209705
Columns: 135 entries, eventid to related
dtypes: datetime64[ns](1), float64(54), int64(23), object(57)
memory usage: 216.0+ MB


In [16]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
eventid,209706.0,2.004867e+11,1.351933e+09,1.970000e+11,1.992080e+11,2.012010e+11,2.015123e+11,2.020123e+11
iyear,209706.0,2.004801e+03,1.351932e+01,1.970000e+03,1.992000e+03,2.012000e+03,2.015000e+03,2.020000e+03
imonth,209706.0,6.455285e+00,3.387098e+00,0.000000e+00,4.000000e+00,6.000000e+00,9.000000e+00,1.200000e+01
iday,209706.0,1.552793e+01,8.801104e+00,0.000000e+00,8.000000e+00,1.500000e+01,2.300000e+01,3.100000e+01
extended,209706.0,5.152452e-02,2.210655e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
...,...,...,...,...,...,...,...,...
nreleased,13494.0,-3.194553e+01,6.449431e+01,-1.000000e+02,-9.900000e+01,0.000000e+00,1.000000e+00,2.958000e+03
INT_LOG,209706.0,-4.424289e+00,4.543800e+00,-9.000000e+00,-9.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
INT_IDEO,209706.0,-4.339318e+00,4.640302e+00,-9.000000e+00,-9.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
INT_MISC,209706.0,8.684062e-02,5.369428e-01,-9.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00


In [17]:
df.isna().sum().sort_values(ascending=False).head(15)

gsubname3           209683
weapsubtype4        209636
weapsubtype4_txt    209636
weaptype4           209633
weaptype4_txt       209633
claimmode3          209566
claimmode3_txt      209566
gsubname2           209522
divert              209368
claim3              209297
guncertain3         209296
gname3              209292
ransomnote          209131
attacktype3         209048
attacktype3_txt     209048
dtype: int64

In [21]:
df.columns

Index(['eventid', 'iyear', 'imonth', 'iday', 'approxdate', 'extended',
       'resolution', 'country', 'country_txt', 'region',
       ...
       'addnotes', 'scite1', 'scite2', 'scite3', 'dbsource', 'INT_LOG',
       'INT_IDEO', 'INT_MISC', 'INT_ANY', 'related'],
      dtype='object', length=135)

In [28]:
def clean_gt_df(df):
    cols_to_drop = []
    for col in list(df.columns):
        if df[col].isna().sum() > 20_000:
            cols_to_drop.append(col)
    df = df.drop(columns = cols_to_drop)
    return df

In [29]:
df = clean_gt_df(df)
df.head()

,eventid,iyear,imonth,iday,extended,country,country_txt,region,region_txt,provstate,...,weaptype1_txt,nkill,nwound,property,ishostkid,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY
0,197000000001,1970,7,2,0,58,Dominican Republic,2,Central America & Caribbean,National,...,Unknown,1.0,0.0,0,0.0,PGIS,0,0,0,0
1,197000000002,1970,0,0,0,130,Mexico,1,North America,Federal,...,Unknown,0.0,0.0,0,1.0,PGIS,0,1,1,1
2,197001000001,1970,1,0,0,160,Philippines,5,Southeast Asia,Tarlac,...,Unknown,1.0,0.0,0,0.0,PGIS,-9,-9,1,1
3,197001000002,1970,1,0,0,78,Greece,8,Western Europe,Attica,...,Explosives,NaN,NaN,1,0.0,PGIS,-9,-9,1,1
4,197001000003,1970,1,0,0,101,Japan,4,East Asia,Fukouka,...,Incendiary,NaN,NaN,1,0.0,PGIS,-9,-9,1,1
